# Домашнее задание № 11. Машинный перевод

## Задание 1 (6 баллов + 2 доп балла).
Нужно обучить трансформер на этом же или на другом корпусе (можно взять другую языковую пару с того же сайте) и оценивать его на всей тестовой выборке (а не на 10 примерах как сделал я). 

Чтобы получить 2 доп балла вам нужно будет придумать как оптимизировать функцию translate. Подсказка: модель может предсказывать батчами.


In [1]:
!pip install tokenizers

     |████████████████████████████████| 6.6 MB 5.2 MB/s 


In [2]:
import numpy as np
import tensorflow as tf

from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase
from tokenizers.trainers import WordPieceTrainer
from tokenizers import decoders

from sklearn.model_selection import train_test_split

Скачиваем корпуса из [TED2020](https://opus.nlpl.eu/TED2020.php) (возьмем языковую пару EN-FR).

In [3]:
from google.colab import drive

ROOT_DIR = '/content/drive' 
drive.mount(ROOT_DIR)

Mounted at /content/drive


In [4]:
!wget -P drive/MyDrive/HSE_compling/HW_11 https://opus.nlpl.eu/download.php?f=TED2020/v1/moses/en-fr.txt.zip

--2022-05-22 16:32:27--  https://opus.nlpl.eu/download.php?f=TED2020/v1/moses/en-fr.txt.zip
Resolving opus.nlpl.eu (opus.nlpl.eu)... 193.166.25.9
Connecting to opus.nlpl.eu (opus.nlpl.eu)|193.166.25.9|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://object.pouta.csc.fi/OPUS-TED2020/v1/moses/en-fr.txt.zip [following]
--2022-05-22 16:32:28--  https://object.pouta.csc.fi/OPUS-TED2020/v1/moses/en-fr.txt.zip
Resolving object.pouta.csc.fi (object.pouta.csc.fi)... 86.50.254.18, 86.50.254.19
Connecting to object.pouta.csc.fi (object.pouta.csc.fi)|86.50.254.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30006604 (29M) [application/zip]
Saving to: ‘drive/MyDrive/HSE_compling/HW_11/download.php?f=TED2020%2Fv1%2Fmoses%2Fen-fr.txt.zip’

download.php?f=TED2 100%[===================>]  28.62M  11.0MB/s    in 2.6s    

2022-05-22 16:32:32 (11.0 MB/s) - ‘drive/MyDrive/HSE_compling/HW_11/download.php?f=TED2020%2Fv1%2Fmoses%2Fen-fr.txt.z

In [5]:
!unzip drive/MyDrive/HSE_compling/HW_11/download.php?f=TED2020%2Fv1%2Fmoses%2Fen-fr.txt.zip -d drive/MyDrive/HSE_compling/HW_11

Archive:  drive/MyDrive/HSE_compling/HW_11/download.php?f=TED2020%2Fv1%2Fmoses%2Fen-fr.txt.zip
  inflating: drive/MyDrive/HSE_compling/HW_11/README  
  inflating: drive/MyDrive/HSE_compling/HW_11/LICENSE  
  inflating: drive/MyDrive/HSE_compling/HW_11/TED2020.en-fr.en  
  inflating: drive/MyDrive/HSE_compling/HW_11/TED2020.en-fr.fr  
  inflating: drive/MyDrive/HSE_compling/HW_11/TED2020.en-fr.xml  


Откроем датасет и посмотрим на примеры.

In [6]:
project_dir = 'drive/MyDrive/HSE_compling/HW_11/'

In [7]:
en_sents_all = open(f'{project_dir}TED2020.en-fr.en').read().lower().splitlines()
fr_sents_all = open(f'{project_dir}TED2020.en-fr.fr').read().lower().splitlines()

print(en_sents_all[5], fr_sents_all[5])
print(en_sents_all[-5], fr_sents_all[-5])

print(len(en_sents_all))

(laughter) i flew on air force two for eight years.  j'ai volé avec air force 2 pendant huit ans. 
today, we stand ready to tell our own stories without compromise, without apologies.  aujourd'hui, nous sommes prêts à raconter notre propre histoire, sans faire de compromis, sans s'excuser. 
410443


Сразу отложим тестовую выборку.

In [8]:
en_sents, en_sents_test, fr_sents, fr_sents_test = train_test_split(en_sents_all, fr_sents_all, test_size=0.05)

print(len(en_sents), len(en_sents_test))

389920 20523


Для токенизации используем WordPiece Tokenizer из библиотеки tokenizers от HG. В токенизатор передаем специальные токены: UNK для неизвестных слов, CLS для начала текста, SEP для конца текста, PAD для паддинга.

In [9]:
# токенизация для английского
tokenizer_en = Tokenizer(WordPiece(), )
tokenizer_en.normalizer = normalizers.Sequence([Lowercase()])
tokenizer_en.pre_tokenizer = Whitespace()

trainer_en = WordPieceTrainer(vocab_size=30000, special_tokens=['[UNK]', '[CLS]', '[SEP]', '[PAD]'])
tokenizer_en.train(files=[f'{project_dir}TED2020.en-fr.en'], trainer=trainer_en)

# токенизация для французского
tokenizer_fr = Tokenizer(WordPiece(), )
tokenizer_fr.normalizer = normalizers.Sequence([Lowercase()])
tokenizer_fr.pre_tokenizer = Whitespace()

trainer_fr = WordPieceTrainer(vocab_size=30000, special_tokens=['[UNK]', '[CLS]', '[SEP]', '[PAD]'])
tokenizer_fr.train(files=[f'{project_dir}TED2020.en-fr.fr'], trainer=trainer_fr)

In [10]:
# декодеры из decoders (библиотеки tokenizers)
tokenizer_en.decoder = decoders.WordPiece()
tokenizer_fr.decoder = decoders.WordPiece()

Соохраним токенизаторы в отдельные файлы, чтобы не потерять их.

In [11]:
tokenizer_en.save(f'{project_dir}tokenizer_en')
tokenizer_fr.save(f'{project_dir}tokenizer_fr')

tokenizer_en = Tokenizer.from_file(f'{project_dir}tokenizer_en')
tokenizer_fr = Tokenizer.from_file(f'{project_dir}tokenizer_fr')

Переводим тексты в последовательности индексов (в начало добавляется токен CLS, в конец -- токен SEP).

In [12]:
def encode(text, tokenizer, target=False):
    return [tokenizer.token_to_id('[CLS]')] + tokenizer.encode(text).ids + [tokenizer.token_to_id('[SEP]')]

X_en = [encode(text, tokenizer_en) for text in en_sents]
X_fr = [encode(text, tokenizer_fr, target=True) for text in fr_sents]

Посмотрим на среднюю длину последовательностей в обоих корпусах.

In [13]:
max_len_en = np.mean([len(x) for x in X_en])
max_len_fr = np.mean([len(x) for x in X_fr])

print(max_len_en, max_len_fr)

22.84078528929011 24.870665777595406


Оставим в качестве значений max_len_en и max_len_fr значения 22 и 24 соответственно.

In [14]:
max_len_en, max_len_fr = 22, 24

In [15]:
PAD_IDX = tokenizer_en.token_to_id('[PAD]')

In [16]:
X_en = tf.keras.preprocessing.sequence.pad_sequences(
      X_en, maxlen=max_len_en, padding='post', value=PAD_IDX)

X_fr_out = tf.keras.preprocessing.sequence.pad_sequences(
      [x[1:] for x in X_fr], maxlen=max_len_fr-1, padding='post', value=PAD_IDX)

X_fr_dec = tf.keras.preprocessing.sequence.pad_sequences(
      [x[:-1] for x in X_fr], maxlen=max_len_fr-1, padding='post', value=PAD_IDX)

X_en.shape, X_fr_out.shape, X_fr_dec.shape

((389920, 22), (389920, 23), (389920, 23))

In [17]:
X_en_train, X_en_valid, X_fr_dec_train, X_fr_dec_valid, X_fr_out_train, X_fr_out_valid = train_test_split(X_en, X_fr_dec, X_fr_out, test_size=0.1)

Обучение модели.

In [18]:
def scaled_dot_product_attention(query, key, value, mask):
    """Calculate the attention weights. """
    matmul_qk = tf.matmul(query, key, transpose_b=True)

    # scale matmul_qk
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)

    # add the mask to zero out padding tokens
    if mask is not None:
        logits += (mask * -1e9)

    # softmax is normalized on the last axis (seq_len_k)
    attention_weights = tf.nn.softmax(logits, axis=-1)

    output = tf.matmul(attention_weights, value)

    return output

In [19]:
class MultiHeadAttention(tf.keras.layers.Layer):

    def __init__(self, d_model, num_heads, name='multi_head_attention'):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)

        self.dense = tf.keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(
            inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs['value'], inputs['mask']
        batch_size = tf.shape(query)[0]

        # linear layers
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # split heads
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # scaled dot-product attention
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        # concatenation of heads
        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        # final linear layer
        outputs = self.dense(concat_attention)

        return outputs

In [20]:
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, PAD_IDX), tf.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, tf.newaxis, tf.newaxis, :] 

In [21]:
def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)

In [22]:
class PositionalEncoding(tf.keras.layers.Layer):

    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)

    def call(self, inputs):

        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [23]:
def encoder_layer(units, d_model, num_heads, dropout, name='encoder_layer'):
    inputs = tf.keras.Input(shape=(None, d_model), name='inputs')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    attention = MultiHeadAttention(
      d_model, num_heads, name='attention')({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
    attention = tf.keras.layers.Dropout(rate=dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

    return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [24]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name='encoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='encoder_layer_{}'.format(i),
        )([outputs, padding_mask])

    return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [25]:
def decoder_layer(units, d_model, num_heads, dropout, name='decoder_layer'):
    inputs = tf.keras.Input(shape=(None, d_model), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    attention1 = MultiHeadAttention(
      d_model, num_heads, name='attention_1')(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })
    attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

    attention2 = MultiHeadAttention(
      d_model, num_heads, name='attention_2')(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })
    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [26]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name='decoder'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
    padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [27]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                max_len,
                name='transformer'):
    inputs = tf.keras.Input(shape=(None,), name='inputs')
    dec_inputs = tf.keras.Input(shape=(None,), name='dec_inputs')

    enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)
    # mask the future tokens for decoder inputs at the 1st attention block
    look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)
    # mask the encoder outputs for the 2nd attention block
    dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

    enc_outputs = encoder(
      vocab_size=vocab_size[0],
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      max_len=max_len[0],
    )(inputs=[inputs, enc_padding_mask])

    dec_outputs = decoder(
      vocab_size=vocab_size[1],
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout,
      max_len=max_len[1],
    )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

    outputs = tf.keras.layers.Dense(units=vocab_size[1], name='outputs')(dec_outputs)

    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [28]:
L  = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')

def loss_function(y_true, y_pred):
    loss = L(y_true, y_pred)

    mask = tf.cast(tf.not_equal(y_true, PAD_IDX), tf.float32)
    loss = tf.multiply(loss, mask)

    return tf.reduce_mean(loss)

In [29]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [30]:
tf.keras.backend.clear_session()

NUM_LAYERS = 2
D_MODEL = 256
NUM_HEADS = 8
UNITS = 512
DROPOUT = 0.1

mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    model = transformer(
        vocab_size=(tokenizer_en.get_vocab_size(), tokenizer_fr.get_vocab_size()),
        num_layers=NUM_LAYERS,
        units=UNITS,
        d_model=D_MODEL,
        num_heads=NUM_HEADS,
        dropout=DROPOUT,
        max_len=[max_len_en, max_len_fr])

    optimizer = tf.keras.optimizers.Adam(
        0.001, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

    def accuracy(y_true, y_pred):
        return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

    model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])
    checkpoint = tf.keras.callbacks.ModelCheckpoint(f'{project_dir}model_enfr',
                                                monitor='val_loss',
                                                verbose=1,
                                                save_weights_only=True,
                                                save_best_only=True,
                                                mode='min',
                                                save_freq='epoch')

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


In [31]:
model.fit((X_en_train, X_fr_dec_train), X_fr_out_train, 
             validation_data=((X_en_valid, X_fr_dec_valid), X_fr_out_valid),
             batch_size=200,
             epochs=100,
             callbacks=[checkpoint])

Epoch 1/100
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
1755/1755 [==============================] - ETA: 0s - loss: 2.7215 - accuracy: 0.3316
Epoch 1: val_loss improved from inf to 1.87981, saving model 

KeyboardInterrupt: ignored

Функция для перевода текста (с семинара).

In [32]:
def translate(text):
    input_ids = encode(text.lower(), tokenizer_en)

    input_ids = tf.keras.preprocessing.sequence.pad_sequences([input_ids], maxlen=max_len_en, padding='post', value=PAD_IDX)

    output_ids = [tokenizer_fr.token_to_id('[CLS]')]
    
    pred = model((input_ids, tf.cast([output_ids], tf.int32)), training=False)
 
    while pred.numpy().argmax(2)[0][-1] not in [tokenizer_fr.token_to_id('[SEP]')]:
        if len(output_ids) > max_len_fr:
            break
        output_ids.append(pred.numpy().argmax(2)[0][-1])
        pred = model((input_ids, tf.cast([output_ids], tf.int32)), training=False)

    return tokenizer_fr.decode(output_ids[1:])

In [33]:
translate('Oh my God! They killed Kenny!')

'oh mon dieu! ils ont tué kenny!'

In [34]:
translate('Machine translation is the use of computers to translate from one language to another.')

"la traduction automatique est l'utilisation des ordinateurs pour traduire d'une langue à l'autre."

Функция для перевода текстов батчами.

In [35]:
def translate_batch(texts):

    input_ids = [encode(text.lower(), tokenizer_en) for text in texts]

    input_ids = tf.keras.preprocessing.sequence.pad_sequences(
                        input_ids, 
                        maxlen=max_len_en, 
                        padding='post', 
                        value=PAD_IDX)

    output_ids = [[tokenizer_fr.token_to_id('[CLS]')] for i in range(len(texts))]

    pred = model((input_ids, tf.cast(output_ids, tf.int32)), training=False)

    SEP_IDX = tokenizer_fr.token_to_id('[SEP]')

    # проходимся по output_ids столько раз, сколько позволяет максимальная длина последовательностей (max_len_fr)
    for i in range(max_len_fr-1):
        # проходимся по каждой последовательности
        for i in range(len(output_ids)):
            # если последний индекс в предложении - индекс паддинга или если следующий предсказываемый индекс - индекс окончания предложения,
            # то добавляем в последовательность индекс паддинга
            if output_ids[i][-1] == PAD_IDX or pred.numpy().argmax(2)[i][-1] == SEP_IDX:
                output_ids[i].append(PAD_IDX)
            # в противном случае добавляем следующий предсказываемый моделью индекс
            else:
                output_ids[i].append(pred.numpy().argmax(2)[i][-1])
        # делаем следующее предсказание
        pred = model((input_ids, tf.cast(output_ids, tf.int32)), training=False)

    translated = []
    # декодируем каждую последовательность в output_ids
    for output in output_ids:
        # из последовательности для декодирования оставляем все индексы, кроме 1 (индекса начала предложения) и индексов паддинга
        seq = [idx for idx in output if (idx != PAD_IDX) & (idx != 1)]
        translated.append(tokenizer_fr.decode(seq))

    return translated

In [36]:
translate_batch(['Oh my God! They killed Kenny!', 'Machine translation is the use of computers to translate from one language to another.', 'London is the capital of Great Britain.'])

['oh mon dieu! ils ont tué kenny!',
 "la traduction automatique est l'utilisation des ordinateurs pour traduire d'une langue à l'autre.",
 'londres est le capital de grande - bretagne.']

Оценим модель на тестовой выборке.

In [37]:
print(en_sents_test[0])
print(fr_sents_test[0])

435 representatives, plus 100 senators, and 3 electors from the district of columbia. 
435 représentants, 100 sénateurs et 3 électeurs du district de columbia. 


In [50]:
import nltk
import tqdm

In [57]:
# разделим выборку на батчи
def split_batches(sents, batch_size):
    for i in range(0, len(sents), batch_size):
        yield sents[i:i+batch_size]

en_batches = list(split_batches(en_sents_test, 16))
fr_batches = list(split_batches(fr_sents_test, 16))

In [58]:
bleus = []

for i_b, batch in tqdm.tqdm(enumerate(en_batches)):
    translations = translate_batch(batch)
    for i_t, translation in enumerate(translations):
          reference = tokenizer_fr.encode(translation).tokens
          hypothesis = tokenizer_fr.encode(fr_batches[i_b][i_t]).tokens
          bleus.append(nltk.translate.bleu_score.sentence_bleu([reference], hypothesis))

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
414it [44:02,  6.38s/it]


KeyboardInterrupt: ignored

In [59]:
(sum(bleus)/len(bleus))*100

36.239878522818


## Задание 2 (2 балла).
Прочитайте главу про машинный перевод у Журафски и Маннига - https://web.stanford.edu/~jurafsky/slp3/10.pdf 
Ответьте своими словами в чем заключается техника back translation? Для чего она применяется и что позволяет получить? Опишите по шагам как его применить к паре en-ru на данных из семинара. 

Техника back translation -- это способ увеличить объем обучающих данных путем искусственного создания пар предложений на двух языках.  Back translation применяют в том случае, когда объема имеющегося параллельного корпуса недостаточно, текстов на языке-источнике мало, а текстов на целевом языке много. Суть техники состоит в том, что сначала на параллельном корпусе обучают модель-переводчик с целевого языка на язык-источник, затем с помощью этой модели переводят любые другие тексты на целевом языке на язык-источник, пополняя тем самым объем обучающей выборки для обучения модели-переводчика с языка-источника на целевой язык. 

Применение этой техники к паре en-ru на данных из семинара выглядело бы так:

1) Обучение на имеющемся параллельном корпусе (в нашем случае - из корпуса opus-100) модели, осуществляющей перевод с русского на английский (то же самое, что было проделано на семинаре, только таргетным языком был бы английский);

2) Генерация перевода какого-нибудь другого корпуса на русском языке (например, корпуса Википедии) с помощью данной модели на английский язык; сохранение полученных пар предложений в отдельный датасет;

3) Обучение на полученном датасете MT-модели для перевода с английского языка на русский.